Extract Documents from database and generate questions based on chunk number

In [10]:
from sqlalchemy import select

from generate_q_a_pairs import generate_question_answer_pair, write_question_answer_json_to_csv
from src.db.models.document import Document
from src.app_config import app_config


with app_config.db_session() as db_session:    
    question_gen_selection = input("Generate questions by chunk or document?")
    documents = db_session.execute(select(Document)).scalars().all()
    fields = ["question", "answer", "document_name", "document_source"]
    for document in documents:
        chunk_list= document.chunk
        if question_gen_selection == "chunk":
            for chunk in chunk_list:
                chunk_q_a_json = generate_question_answer_pair(document=chunk.content, num_of_chunks=chunk.num_splits)
                write_question_answer_json_to_csv("question_answer_pairs.csv", fields, chunk_q_a_json)        
        else:
            document_q_a_json = generate_question_answer_pair(document=document, num_of_chunks=len(document.chunks))
            write_question_answer_json_to_csv("question_answer_pairs.csv", fields, document_q_a_json)        
        
        

ValidationError: 3 validation errors for PostgresDBConfig
DB_HOST
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
DB_NAME
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
DB_USER
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing